In [64]:
import requests
import json
import pandas as pd
import bibtexparser
import pprint

# Postgres python
import psycopg2
from psycopg2.extras import Json

def doi2dict(doi):
    #create url
    url = "http://dx.doi.org/" + doi
    
    #create dictionary of http bibtex headers that requests will retrieve from the url
    headers = {"accept": "application/x-bibtex"}
    
    #reqeusts information specified by bibtex from url
    r = requests.get(url, headers = headers).text    
    
    #parse the returned bibtex text to a dictionary
    #NOTE: USE bibtexparser.customization to split strings into list, etc. (https://bibtexparser.readthedocs.io/en/master/bibtexparser.html?highlight=bparser#module-bibtexparser.bparser)
    bibdata = bibtexparser.bparser.BibTexParser().parse(r)
    
    # # print doi metadata
    # pp = pprint.PrettyPrinter(indent=4)
    # pp.pprint(bibdata.entries[0])
    
    #return dict of metadata
    return bibdata.entries[0]

doi = '10.1021/acsami.1c20994'
doi2 = '10.1021/acscentsci.9b00476'

doidict = doi2dict(doi)
doidict2 = doi2dict(doi2)

#create json file with corresponding doi id, dump info into file.
with open(doidict['ID'] + '.json', 'w+') as outfile:
    json.dump(doidict, outfile)

In [66]:
json_list = [doidict, doidict2]
json_list

[{'journal': '{ACS} Applied Materials {\\&}amp$\\mathsemicolon$ Interfaces',
  'title': 'The Solution is the Solution: Data-Driven Elucidation of Solution-to-Device Feature Transfer for $\\uppi$-Conjugated Polymer Semiconductors',
  'author': 'Connor P. Callaway and Aaron L. Liu and Rahul Venkatesh and Yulong Zheng and Myeongyeon Lee and J. Carson Meredith and Martha Grover and Chad Risko and Elsa Reichmanis',
  'pages': '3613--3620',
  'number': '3',
  'volume': '14',
  'publisher': 'American Chemical Society ({ACS})',
  'month': 'jan',
  'year': '2022',
  'url': 'https://doi.org/10.1021%2Facsami.1c20994',
  'doi': '10.1021/acsami.1c20994',
  'ENTRYTYPE': 'article',
  'ID': 'Callaway_2022'},
 {'journal': '{ACS} Central Science',
  'title': '{BigSMILES}: A Structurally-Based Line Notation for Describing Macromolecules',
  'author': 'Tzyy-Shyang Lin and Connor W. Coley and Hidenobu Mochigase and Haley K. Beech and Wencong Wang and Zi Wang and Eliot Woods and Stephen L. Craig and Jeremia

In [72]:
conn = psycopg2.connect(
   database="testdb", 
   user = "postgres", 
   password = "password", 
   host = "127.0.0.1",
   # host = "130.207.79.186",
   # host = "192.168.1.21",
   port = "5432")

print("Connection successful")

cur = conn.cursor()
cur.execute(
    '''
    CREATE TABLE IF NOT EXISTS LITERATURE (
        id SERIAL PRIMARY KEY,
        doi JSONB UNIQUE
    );
      '''
)
for doc in json_list:
    query = """
    INSERT INTO LITERATURE(doi) 
    VALUES(%s)
    ON CONFLICT (doi)
    DO NOTHING;
    """ % Json(doc) #use the psycopg2.Json object to convert to a readable json
    try:
        cur.execute(query)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cur.close()

# cur.execute("SELECT * from LITERATURE")
# rows = cur.fetchall()
# pp = pprint.PrettyPrinter(indent=4)
# pp.pprint(rows)

conn.commit()
print("Operation successful")
conn.close()

Connection successful
Operation successful
